# Step 1: Data preparation using sqrt and z-score

**Goal of the step:** Prepare the data for exploration and modeling.

**Step overview:**
1) Loading of the dataframe
2) Translation of the data from dutch to english
3) Remotion of unavailable data
4) Preparation of the data

Description of the features:

- n_grid: univocal code for each cell in the grid

- pop: number of inhabitants in the cell on the 1st January 2018

- per_for: percentage of residents of whom at least one parent was born abroad in one of the countries of Africa, Latin America or Asia (excluding Indonesia and Japan) or in Turkey

- dist_road: average distance of all inhabitants in an area to the nearest entrance to a national or provincial road, calculated over the road

- unemp: residents who receive unemployment benefits, a social assistance or assistance-related benefit or an occupational disability benefit. This concerns benefits to residents up to the state pension age. A person who receives several types of benefits is counted once.

- p0_14: percentage of inhabitants under 15 years old

- p65_PL: percentage of inhabitants aged 65 or more

- med_inc: standardized median  income, classified as: Low, Mid, High

In [ ]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
from scipy.stats import skew

## 1. Data loading

In [ ]:
# Data loading

df = geopandas.read_file("data/500x500.json")

In [ ]:
# Subset of the relevant features

df = df[['c28992r500', 'INWONER', 'P_NW_MIG_A', "AFS_OPRIT", "UITKMINAOW", "INW_014", 'INW_1524', 'INW_2544', 'INW_4564', 'INW_65PL', 'M_INKHH', 'geometry']]

## 2. Translation from dutch to english

In [ ]:
# Making the columns english readable

df.columns = ['n_grid', 'pop',  'per_for', 'dist_road',
       'unemp','p0_14',   'p15_24',   'p25_44',   'p45_64',
         'p65_PL',    'med_inc',   'geometry']

In [ ]:
# Making the median income data english readable

df['med_inc'] = df['med_inc'].replace(['onclassificeerbaar','00-20 laag','00-40 laag tot onder midden','20-40 onder midden','20-60 onder midden tot midden','40-60 midden','40-80 midden tot boven midden', '60-80 boven midden','60-100 boven midden tot hoog','80-100 hoog'],['unclassifiable','00-20 low','00-40 low to below mid','20-40 below center','20-60 bottom mid to mid','40-60 mid','40-80 mid to top mid','60-80 above center','60-100 top mid to high','80-100 high'])

## 3. Remotion of unavailable data

All the rows with missing information about population, median income and percentage of foreigners are removed

In [ ]:
# Remotion of rows with no data about population's age

df = df[(df.p0_14 != -99997) & (df.p15_24 != -99997) & (df.p25_44 != -99997) & (df.p45_64 != -99997) & (df.p65_PL != -99997)]

In [ ]:
# Remotion of rows with no data about the percentage of foreigners

df = df[(df.per_for != -99997)]

In [ ]:
# Remotion of rows with no data about median income

df = df[df.med_inc != 'onclassificeerbaar']

In [ ]:
# Remotion of rows with no data about unemployment

df = df[df.unemp != -99997]

## 4. Preparation of data

### Skewness calculation

In [ ]:
# Plot of the features to identify skewness and correlation in the distributions

sns.pairplot(df)

In [ ]:
# Skewness test

print(skew(df['pop'], bias=False))
print(skew(df['per_for'], bias=False))
print(skew(df['dist_road'], bias=False))
print(skew(df['unemp'], bias=False))
print(skew(df['p0_14'], bias=False))
print(skew(df['p65_PL'], bias=False))

All the variables have significant skewness to the right, to handle this a sqrt transformation can be performed 

*Rule of thumb +-0.8 is an acceptable value of skewness*

A sqrt transformation instead of a log transformation has been performed because some data have value equal to zero.

### Early data elaboration and square root transformation

In [ ]:
# Grouping of income categories

df.loc[df.med_inc == '00-20 low', 'med_inc'] = "low"
df.loc[df.med_inc == '00-40 low to below mid', 'med_inc'] = "low"
df.loc[df.med_inc == '20-40 below center', 'med_inc'] = "low"

df.loc[df.med_inc == '20-60 bottom mid to mid', 'med_inc'] = "mid"
df.loc[df.med_inc == '40-60 mid', 'med_inc'] = "mid"
df.loc[df.med_inc == '40-80 mid to top mid', 'med_inc'] = "mid"

df.loc[df.med_inc == '60-80 above center', 'med_inc'] = "high"
df.loc[df.med_inc == '60-100 top mid to high', 'med_inc'] = "high"
df.loc[df.med_inc == '80-100 high', 'med_inc'] = "high"

In [ ]:
df2 = df.copy()

In [ ]:
# Turn the unemployment into a percentage of the population

df['unemp'] = df2['unemp']/df2['pop']

In [ ]:
# Calculate the skewness for the unemployment percentage

print(skew(df['unemp'], bias=False))

The skewness is very high, a sqrt transformation will be performed

In [ ]:
# Transformation of the unemployment percentage with the sqrt function

df['unemp'] = np.sqrt(df['unemp'])

In [ ]:
plt.boxplot(df['unemp'])

In [ ]:
# Calculate the skewness for the unemployment percentage

print(skew(df['unemp'], bias=False))

There is still an important outlier that brings the skewness outside the acceptance value, it will be removed from the analysis

In [ ]:
df = df[(df.unemp != df['unemp'].max())]

In [ ]:
# Calculate the skewness for the unemployment percentage

print(skew(df['unemp'], bias=False))

Now the skewness is acceptable

In [ ]:
df['dist_road'] = np.sqrt(df['dist_road'])

In [ ]:
# Calculate the skewness for the distance from the closest provincial/national road

print(skew(df['dist_road'], bias=False))

The value is acceptable

In [ ]:
df['pop'] = np.sqrt(df['pop'])

In [ ]:
# Calculate the skewness for the population

print(skew(df['pop'], bias=False))

The value is acceptable

In [ ]:
# Transformation of the age data into percentages

df['p0_14'] = 100 * df2['p0_14']/(df2['p0_14']+df2['p15_24']+df2['p25_44']+df2['p45_64']+df2['p65_PL'])
df['p15_24'] = 100 * df2['p15_24']/(df2['p0_14']+df2['p15_24']+df2['p25_44']+df2['p45_64']+df2['p65_PL'])
df['p25_44'] = 100 * df2['p25_44']/(df2['p0_14']+df2['p15_24']+df2['p25_44']+df2['p45_64']+df2['p65_PL'])
df['p45_64'] = 100 * df2['p45_64']/(df2['p0_14']+df2['p15_24']+df2['p25_44']+df2['p45_64']+df2['p65_PL'])
df['p65_PL'] = 100 * df2['p65_PL']/(df2['p0_14']+df2['p15_24']+df2['p25_44']+df2['p45_64']+df2['p65_PL'])

In [ ]:
# Plot of the distributions to identify skewness and correlation

sns.pairplot(df)

In [ ]:
print(skew(df['p0_14'], bias=False))
print(skew(df['p65_PL'], bias=False))

In [ ]:
plt.boxplot(df['p65_PL'])

After the percentage transformation, the skewness value for the population between 0 and 14 years old is in the acceptance range, while the population between 65 and older has still significant skewness.

From the boxplot, we can see that there are several outliers that influence the skewness of the distribution, these values will be pushed to the 0.9 quantile of the distribution.

In [ ]:
# Outliers management percentage of population 65 years old and older

p65pll = (df['p65_PL'].quantile(0.10))
p65plu = (df['p65_PL'].quantile(0.90))

In [ ]:
# Adjusting outliers for the percentage of population 65 years old and older

df["p65_PL"] = np.where(df["p65_PL"] < p65pll, p65pll, df['p65_PL'])
df["p65_PL"] = np.where(df["p65_PL"] > p65plu, p65plu, df['p65_PL'])

In [ ]:
print(skew(df['p65_PL'], bias=False))

Now the skewness is in the acceptance range, the level of skewness is down from 1.08 to 0.58

In [ ]:
df['per_for'] = np.sqrt(df['per_for'])

In [ ]:
print(skew(df['per_for'], bias=False))

The skewness for the percentage of foreigners is reduced from 0.97 to 0.26, now acceptable

### Z-Score transformation

In [ ]:
# Standardization of the population data

df['pop'] = (df['pop'] - df['pop'].mean())/df['pop'].std()

In [ ]:
# Standardization of the distance from the closest provincial road data

df['dist_road'] = (df['dist_road'] - df['dist_road'].mean())/df['dist_road'].std()

In [ ]:
# Standardization of unemployment data

df['unemp'] = (df['unemp'] - df['unemp'].mean())/df['unemp'].std()

In [ ]:
# Standardization of the age percentages

df['p0_14'] = (df['p0_14'] - df['p0_14'].mean())/df['p0_14'].std()
df['p65_PL'] = (df['p65_PL'] - df['p65_PL'].mean())/df['p65_PL'].std()

In [ ]:
df[['pop', 'per_for', 'dist_road', 'unemp', 'p0_14', 'p65_PL']].boxplot(figsize=(10,4))

In [ ]:
# Outliers management for the percentage of the population aged 14 or less

p014l = (df['p0_14'].quantile(0.10))
p014u = (df['p0_14'].quantile(0.90))

In [ ]:
# Adjusting the outliers

df["p0_14"] = np.where(df["p0_14"] < p014l, p014l, df['p0_14'])
df["p0_14"] = np.where(df["p0_14"] > p014u, p014u, df['p0_14'])

In [ ]:
df.to_csv('data/grid.csv', index = False)